# SWECRIS DBstep 1: Data Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd
from pathlib import Path
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

In [2]:
import re
import string
import pprint

In [3]:
!pip install stanza
!pip install spacy-stanza

/bin/bash: pip: command not found
/bin/bash: pip: command not found


In [4]:
import stanza
from spacy_stanza import StanzaLanguage

- Global variables

In [5]:
svStanzaDownloaded = False
snlpInitialized = False
nlpStanza = {}

pp = pprint.PrettyPrinter(indent=2, width=200)

categories = []

- Needed to be able to run on __gpu (cuda)__

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

- Either load data from local drive or download first from google drive, if within docker container
    check this great link [__Colab__](https://colab.research.google.com/)  
    the alternative is [__Binder__](https://mybinder.org) though there is no gpu there

In [7]:
table_name = 'swecris'
db_path = 'db/categorised_swecris.sqlite'

if not Path(db_path).is_file():
    gdd.download_file_from_google_drive(
        file_id='1O8A7QOpynNTsblDHve0R7xD0CQFQzxrx',
        dest_path=db_path,
    )
    print('Dowloaded from google drive')
db_path = 'sqlite:///' + db_path

In [8]:
!pip install --upgrade gupload

/bin/bash: pip: command not found


In [9]:
from pydrive.auth import GoogleAuth
from google.colab import auth
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

### For uploading files to googl drive

First you have to share a google drive folder in __edit__ mode, for example: https://drive.google.com/drive/folders/1wNJUZtQD_6oBvvUhWRlAb0xjjjoLZQ?usp=sharing (the link is not valid, it's just an example)  
The id you need to copy into the code is: 1wNJUZtQD_6oBvvUhWRlAb0xjjjoLZQ  
see https://stackoverflow.com/questions/64808087/how-do-i-save-files-from-google-colab-to-google-drive

```python
!pip install --upgrade gupload

from pydrive.auth import GoogleAuth
from google.colab import auth

# Authenticate and create the PyDrive client.
auth.authenticate_user()

# upload one file (res.png)
!gupload --to '1wNJUZtQD_6oBvvUhWRlAb0xjjjoLZQ' res.png

# if multiple images
!gupload --to '1wNJUZtQD_6oBvvUhWRlAb0xjjjoLZQ' *.png
```

__OR__


Easy way to save is to mount google drive and provide path to the folder in drive where files need to be saved.
Mount Drive

```python
from google.colab import drive
drive.mount('/content/drive')

# Generate Random Image and show

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

rgb_img = np.random.randint(0, 255, size=(400,400,3), dtype=np.uint8)

#cv2.imshow('RGB', rgb_img)
#cv2.waitKey(0)

cv2_imshow(rgb_img)

#Save image to drive directly

#Path to save in drive can be found easily by clicking files on left side, navigating to folder and right click to choose Copy path. This will save the image to folder named Delete in Google Drive.

# Save Image
cv2.imwrite('/content/drive/My Drive/Delete/res.png' , rgb_img)
```

#### Using copy command

- This was not asked but can be useful. Assuming /content/res.png is image path and /content/drive/My Drive/Delete is destination.

-  Alternate option is write image in colab and copy to drive using

`!cp "/content/res.png" "/content/drive/My Drive/Delete"`

after drive mount.

Path for sglite daatabase: `/content/db/categorised_swecris.sqlite`

In [10]:
engine = create_engine(db_path, echo=True)
data_df = pd.read_sql_table(table_name, engine)
data_df.set_index('index')

2020-12-10 11:13:35,943 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-10 11:13:35,944 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:13:35,945 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-10 11:13:35,946 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:13:35,948 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-10 11:13:35,949 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:13:35,950 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='view' ORDER BY name
2020-12-10 11:13:35,950 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:13:35,952 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("swecris")
2020-12-10 11:13:35,953 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:13:35,956 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION AL

,diarienr,title,abstract,organization,Formas1,Self1
index,,,,,,
0,2018-04872,The Agenda 2030 Compass Methodology and toolbo...,Syfte och mål: Syftet är att utveckla Agenda 2...,Vinnova,,1
1,2018-04826,IP CHECK,Syfte och mål: Vi har lyckats skapa processer ...,Vinnova,,1
2,2019-02128,Innovativa lösningar för äldres oberoende,Syfte och mål: Målet med projektet är att utve...,Vinnova,,2
3,2017-05317,Arena Research2Business,Syfte och mål: Stärka kommersiell samverkan oc...,Vinnova,,2
4,2019-03926,Hollbium AB,Syfte och mål: Vårt mål är att tjänstefiera ko...,Vinnova,,2
...,...,...,...,...,...,...
5618,2016-02089,2017 års konferens om Earth System Governance,Den åttonde årliga konferensen inom projektet ...,Formas,17,
5619,2016-00872,Expertkommitteer i gränslandet mellan forsknin...,Forskning används allt oftare som grund i poli...,Formas,17,
5620,2016-01535,Framtida människor och hållbarhetsbegreppet,Begreppet hållbarhet har kommit att få stor be...,Formas,17,


## Data cleaning

1. Clean text round 1
    - transform to lower case
    - replace punctuation with spaces
2. Clean text round 2
    - remove new lines (replace with a single space)
    - remove some punctuation and non-sensical text that was missed in round 1
3. Lemmatize text


### For google
- Needed for JupyterLab within docker container on google cloud, by default these packages are not installed
- On binder packages are preinstalled by listing them in the `requirements.txt`

In [11]:
!pip install stanza
!pip install spacy-stanza

/bin/bash: pip: command not found
/bin/bash: pip: command not found


In [12]:
def initStanzaPipeline(lang):
    downloadStanza(lang)
    global snlpInitialized
    global nlpStanza
    if not snlpInitialized:
        snlp = stanza.Pipeline(lang=lang)
        nlpStanza['snlp'] = StanzaLanguage(snlp)
        snlpInitialized = True

In [13]:
def removeNewlines(text):
    text = re.sub(r'\\r\\n', ' ', text)
    text = re.sub(r'\\n', ' ', text)
    return ' '.join(text.split())

In [14]:
def clean_text_round1(text):
    """First round of cleaning.

    Make text lowercase, remove text in square brackets,
    remove punctuation and remove words containing numbers.
    """
    text = removeNewlines(text)
    text = text.lower()
    text = re.sub(r'|[.*?]|', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

In [15]:
data_clean = pd.DataFrame(data_df.abstract.apply(clean_text_round1))

In [16]:
print(data_clean.abstract)

0       syfte och mål syftet är att utveckla agenda  d...
1       syfte och mål vi har lyckats skapa processer o...
2       syfte och mål målet med projektet är att utvec...
3       syfte och mål stärka kommersiell samverkan och...
4       syfte och mål vårt mål är att tjänstefiera kon...
                              ...                        
5618    den åttonde årliga konferensen inom projektet ...
5619    forskning används allt oftare som grund i poli...
5620    begreppet hållbarhet har kommit att få stor be...
5621    i arbetet med att möta dagens miljöproblem har...
5622    överenskommelsen mellan fns medlemsstater krin...
Name: abstract, Length: 5623, dtype: object


In [20]:
def clean_text_round2(text):
    """Second round of cleaning.

    Get rid of some additional punctuation and non-sensical text
    that was missed the first time around.
    """
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[‘’“”…–|-]', '', text)
    text = re.sub(r'\s\s+', ' ', text)
    text = re.sub(r'syfte och mål', '', text)
    text = re.sub(r'mål|syftet|syfte', '', text)
    text = re.sub(r'konferensen |konferens |projektet |projekt ', '', text)
    return text

In [21]:
data_clean = pd.DataFrame(data_clean.abstract.apply(clean_text_round2))

In [22]:
print(data_clean.abstract)

0        är att utveckla agenda dvs metodik och stödve...
1        vi har lyckats skapa processer och strukturer...
2        et med är att utveckla lösningar för äldres o...
3        stärka kommersiell samverkan och öka kontakte...
4        vårt är att tjänstefiera koncept där inomhuso...
                              ...                        
5618    den åttonde årliga inom earth system governanc...
5619    forskning används allt oftare som grund i poli...
5620    begreppet hållbarhet har kommit att få stor be...
5621    i arbetet med att möta dagens miljöproblem har...
5622    överenskommelsen mellan fns medlemsstater krin...
Name: abstract, Length: 5623, dtype: object


In [23]:
def downloadStanza(lang):
    # download sv stanza (to avoid downloading it for every doc)
    global svStanzaDownloaded
    if not svStanzaDownloaded:
        stanza.download(lang)
        svStanzaDownloaded = True

In [24]:
def lemmatizeText(text):
    initStanzaPipeline('sv')
    doc = nlpStanza['snlp'](text)
    lemmatized_text = ''
    for token in doc:
        if token.lemma_ == 'all':
            token.lemma_ = 'alla'
        lemmatized_text += token.lemma_ + ' '
    text = lemmatized_text
    return text

In [25]:
data_clean = pd.DataFrame(data_clean.abstract.apply(lemmatizeText))

2020-12-10 11:15:20 INFO: Downloading default packages for language: sv (Swedish)...
2020-12-10 11:15:20 INFO: File exists: /home/nesko/stanza_resources/sv/default.zip.
2020-12-10 11:15:24 INFO: Finished downloading models and saved to /home/nesko/stanza_resources.
2020-12-10 11:15:24 INFO: Loading these models for language: sv (Swedish):
| Processor | Package   |
-------------------------
| tokenize  | talbanken |
| pos       | talbanken |
| lemma     | talbanken |
| depparse  | talbanken |

2020-12-10 11:15:24 INFO: Use device: gpu
2020-12-10 11:15:24 INFO: Loading: tokenize
2020-12-10 11:15:30 INFO: Loading: pos
2020-12-10 11:15:31 INFO: Loading: lemma
2020-12-10 11:15:31 INFO: Loading: depparse
2020-12-10 11:15:33 INFO: Done loading processors!


In [27]:
data_df = data_df.drop(columns=['abstract'])
data_df

,index,diarienr,title,organization,Formas1,Self1
0,0,2018-04872,The Agenda 2030 Compass Methodology and toolbo...,Vinnova,,1
1,1,2018-04826,IP CHECK,Vinnova,,1
2,2,2019-02128,Innovativa lösningar för äldres oberoende,Vinnova,,2
3,3,2017-05317,Arena Research2Business,Vinnova,,2
4,4,2019-03926,Hollbium AB,Vinnova,,2
...,...,...,...,...,...,...
5618,5618,2016-02089,2017 års konferens om Earth System Governance,Formas,17,
5619,5619,2016-00872,Expertkommitteer i gränslandet mellan forsknin...,Formas,17,
5620,5620,2016-01535,Framtida människor och hållbarhetsbegreppet,Formas,17,
5621,5621,2016-00545,Att skolas till expert. IPCC och IPBES sociali...,Formas,17,


In [29]:
data_clean.rename(columns={"index": "index2"})
data_clean

,abstract
0,vara att utveckla agenda dvs metodik och stö...
1,vi ha lycka skapa processe och struktur som ...
2,et med vara att utveckla lösning för äldres ...
3,stärka kommersiell samverkan och öka kontakt...
4,vi vara att tjänstefiera koncept där inomhus...
...,...
5618,en åtta årlig inom earth system governance esg...
5619,forskning använda allt ofta som grund i politi...
5620,begrepp hållbarhet ha komma att få stor betyde...
5621,i arbete med att möta dag miljöproblem ha expe...


In [32]:
data_merged = pd.concat([data_df, data_clean], axis=1)
data_merged

,index,diarienr,title,organization,Formas1,Self1,abstract
0,0,2018-04872,The Agenda 2030 Compass Methodology and toolbo...,Vinnova,,1,vara att utveckla agenda dvs metodik och stö...
1,1,2018-04826,IP CHECK,Vinnova,,1,vi ha lycka skapa processe och struktur som ...
2,2,2019-02128,Innovativa lösningar för äldres oberoende,Vinnova,,2,et med vara att utveckla lösning för äldres ...
3,3,2017-05317,Arena Research2Business,Vinnova,,2,stärka kommersiell samverkan och öka kontakt...
4,4,2019-03926,Hollbium AB,Vinnova,,2,vi vara att tjänstefiera koncept där inomhus...
...,...,...,...,...,...,...,...
5618,5618,2016-02089,2017 års konferens om Earth System Governance,Formas,17,,en åtta årlig inom earth system governance esg...
5619,5619,2016-00872,Expertkommitteer i gränslandet mellan forsknin...,Formas,17,,forskning använda allt ofta som grund i politi...
5620,5620,2016-01535,Framtida människor och hållbarhetsbegreppet,Formas,17,,begrepp hållbarhet ha komma att få stor betyde...
5621,5621,2016-00545,Att skolas till expert. IPCC och IPBES sociali...,Formas,17,,i arbete med att möta dag miljöproblem ha expe...


In [34]:
table_name_cleaned = 'clean_swecris'
data_merged.to_sql(table_name_cleaned, engine, if_exists='fail')

2020-12-10 11:56:49,844 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("clean_swecris")
2020-12-10 11:56:49,845 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:56:49,846 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("clean_swecris")
2020-12-10 11:56:49,847 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:56:49,853 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE clean_swecris (
	level_0 BIGINT, 
	"index" BIGINT, 
	diarienr TEXT, 
	title TEXT, 
	organization TEXT, 
	"Formas1" TEXT, 
	"Self1" TEXT, 
	abstract TEXT
)


2020-12-10 11:56:49,854 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:56:49,867 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-10 11:56:49,868 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_clean_swecris_level_0 ON clean_swecris (level_0)
2020-12-10 11:56:49,869 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 11:56:49,886 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-10 11:56:49,895 INFO sqlalchemy.engine.base.Engine BEGIN (im

## CoutVectrizer

- default ngram_range(1,1) = __unigrams__
- for unigrams and __bigrams__ set ngram_range(1,2)

In [35]:
STOP_WORDS = set(
    """
aderton adertonde adjö aldrig allas allt alltid alltså än andra andras
annan annat ännu artonde arton åtminstone att åtta åttio åttionde åttonde av
även

båda bådas bakom bara bäst bättre behöva behövas behövde behövt beslut beslutat
beslutit bland blev bli blir blivit bort borta bra

då dag dagar dagarna dagen där därför de del delen dem den deras dess det detta
dig din dina dit ditt dock du

efter eftersom elfte eller elva en enkel enkelt enkla enligt er era ert ett
ettusen

få fanns får fått fem femte femtio femtionde femton femtonde fick fin finnas
finns fjärde fjorton fjortonde fler flera flesta följande för före förlåt förra
första fram framför från fyra fyrtio fyrtionde

gå gälla gäller gällt går gärna gått genast genom gick gjorde gjort god goda
godare godast gör göra gott

ha hade haft han hans har här heller hellre helst helt henne hennes hit hög
höger högre högst hon honom hundra hundraen hundraett hur

i ibland idag igår igen imorgon in inför inga ingen ingenting inget innan inne
inom inte inuti

ja jag jämfört

kan kanske knappast kom komma kommer kommit kr kunde kunna kunnat kvar

länge längre långsam långsammare långsammast långsamt längst långt lätt lättare
lättast legat ligga ligger lika likställd likställda lilla lite liten litet

man många måste med mellan men mer mera mest mig min mina mindre minst mitt
mittemot möjlig möjligen möjligt möjligtvis mot mycket

någon någonting något några när nästa ned nederst nedersta nedre nej ner ni nio
nionde nittio nittionde nitton nittonde nödvändig nödvändiga nödvändigt
nödvändigtvis nog noll nr nu nummer

och också ofta oftast olika olikt om oss

över övermorgon överst övre

på

rakt rätt redan

så sade säga säger sagt samma sämre sämst sedan senare senast sent sex sextio
sextionde sexton sextonde sig sin sina sist sista siste sitt sjätte sju sjunde
sjuttio sjuttionde sjutton sjuttonde ska skall skulle slutligen små smått snart
som stor stora större störst stort

tack tidig tidigare tidigast tidigt till tills tillsammans tio tionde tjugo
tjugoen tjugoett tjugonde tjugotre tjugotvå tjungo tolfte tolv tre tredje
trettio trettionde tretton trettonde två tvåhundra

under upp ur ursäkt ut utan utanför ute

vad vänster vänstra var vår vara våra varför varifrån varit varken värre
varsågod vart vårt vem vems verkligen vi vid vidare viktig viktigare viktigast
viktigt vilka vilken vilket vill
""".split()
)

In [36]:
common_stopwords = [
    'både',
    'först',
    'ge',
    'minska',
    'samt',
    'se',
    'synnerhet',
    'sätt',
    'ta',
    'vara',
    'år',
    'öka'
    ]

extra_stopwords = [
    'ansökan', 'avhandling',
    'bakgrund',
    'disputationsämne',
    'huvudsyfte',
    'mål', 'målsättning',
    'konferens',
    'projekt',
    'resebidrag',
    'syfte',
    'undertecknad'
    ]

In [37]:
STOP_WORDS = STOP_WORDS.union(common_stopwords)
STOP_WORDS = STOP_WORDS.union(extra_stopwords)

In [38]:
# cv = CountVectorizer(stop_words=sw.STOP_WORDS, ngram_range=(1, 2), max_features=1000)
cv = CountVectorizer(stop_words=STOP_WORDS, max_features=1000)

In [39]:
data_cv = cv.fit_transform(data_clean.abstract)

In [40]:
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm = data_dtm.transpose()

In [41]:
data_dtm

,0,1,2,3,4,5,6,7,8,9,...,5613,5614,5615,5616,5617,5618,5619,5620,5621,5622
ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
additiv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adressera,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
affär,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
affärsmodell,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
öppen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
öppna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
övergripande,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
övervakning,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


__Find the top 30 words within each abstract__

In [42]:
top_dict = {}
for c in data_dtm.columns:
    top = data_dtm[c].sort_values(ascending=False).head(30)
    top_dict[c] = list(zip(top.index, top.values))

__Print the top 15 words within each abstract__

In [43]:
for name, top_words in top_dict.items():
    print()
    print(', '.join([word for word, count in top_words[0:14]]))
    print('---')


agenda, olik, bidra, fas, resultat, bedömning, myndighet, plattform, alternativ, strategisk, forskning, företag, process, effekt
---

moment, strategi, hjälpa, genomförande, kräva, skapa, immateriell, tillgång, upplägg, identifiering, resultat, förvänta, utveckla, hantering
---

energi, social, deltagande, fysisk, kombinera, lösning, utveckla, livsmedel, et, aktivitet, sverige, tillräcklig, projektgrupp, hos
---

näringsliv, forskning, insats, företag, kommersialisering, aktivitet, samhälle, testa, utveckla, område, stärka, nationell, gemensam, samverkan
---

skapa, genomförande, feedback, resultat, manuell, användning, koncept, arbeta, arbete, förvänta, projektledning, effekt, produktion, uppnå
---

fas, uppfylld, förvänta, program, upplägg, anställd, ske, sån, arbete, denna, positiv, effekt, arbeta, genomförande
---

cirkulär, visa, värdekedja, genomföra, hantering, digitalisering, uppskalning, processe, potential, regelverk, vanlig, hel, förvänta, genomförande
---

basera, ai, resu